In [ ]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
def normalize_landmarks(landmarks):
    x_values = []
    y_values = []
    for landmark in landmarks:
    #if landmark.visibility >= 0.50:
        x_values.append(landmark.x)
        y_values.append(landmark.y)
    
    min_x = min(x_values)
    min_y = min(y_values)
    normalized_x  = [x - min_x for x in x_values]

    normalized_y = [y- min_y for y in y_values]
    #normalized_landmarks = list(zip(normalized_x, normalized_y))
    normalized_landmarks = [item for pair in zip(normalized_x, normalized_y) for item in pair]
    #print(normalized_landmarks)
    return normalized_landmarks
    


In [ ]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #image = cv2.resize(image, 480,640)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        try:
            landmarks = results.pose_landmarks.landmark
            print(normalize_landmarks(landmarks))
            
            #print(image.shape)
            #print(topandbottom(landmarks))
            #topandbottom(landmarks,resolution)
        except:
            #print(e)
            pass
        
        #print(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value])
          
        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        
    



    cap.release()
    cv2.destroyAllWindows()


## normalize landmarks

In [ ]:
import csv

def export_landmark(normalized_landmarks, action):
    #print(type(normalized_landmarks))
    # Check for None values in normalized_landmarks
    if any(landmark is None for landmark in normalized_landmarks):
        print('There are None values in normalized_landmarks.')
        return

    # Flatten the normalized_landmarks list
    #flat_normalized_landmarks = [coord for landmark in normalized_landmarks for coord in landmark]

    #print(len(flat_normalized_landmarks))
    try:
        # Insert the action at the beginning of the flattened list
        #flat_normalized_landmarks.insert(0, action)
        normalized_landmarks.insert(0,action)
    except TypeError as e:
        print(f"Error: {e}")
        print(f"Action: {action}")
        print(f"Type of action: {type(action)}")
        print(f"Normalized landmarks: {normalized_landmarks}")
        print(f"Type of normalized landmarks: {type(normalized_landmarks)}")
        return

    # Print the type of flat_normalized_landmarks
    #print(type(flat_normalized_landmarks))

    # Write the row to the CSV file
    with open('try2squatswideornarrow.csv', mode='a', newline='') as f:
        csvwriter = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        #csvwriter.writerow(flat_normalized_landmarks)
        csvwriter.writerow(normalized_landmarks)
'''
# Example usage
normalized_landmarks_example = [[0.1, 0.2, 0.3], [0.4, 0.5, 0.6]]
action_example = "example_action"
export_landmark(normalized_landmarks_example, action_example)
'''

## MAKE NEW CSV FOR COORDS

In [ ]:
import csv
landmarks = ['class']
for val in range (1,33+1):
    landmarks += [f'x{val}',f'y{val}']
print(landmarks)
with open('try2squatswideornarrow.csv', mode='w', newline='') as f:
    csvwriter = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csvwriter.writerow(landmarks)

## Export landmarks for squats

In [ ]:
## exporting landmarks
import csv
def export_landmark(normalized_landmarks,action):
    #print('started exporting')
    if any(landmark is None for landmark in normalized_landmarks):
        print('There are None values in normalized_landmarks.')
        return
    flat_normalized_landmarks = [coord for landmark in normalized_landmarks for coord in landmark]
    print(type(flat_normalized_landmarks))
    if None in flat_normalized_landmarks:
        print('none values there')
    row = flat_normalized_landmarks.insert(0,action)
    with open('try2squatswideornarrow.csv', mode='a', newline='') as f:
        csvwriter = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csvwriter.writerow(row)
    print('exported')

## code to export coordiantes based on keys pressed

In [ ]:
cap = cv2.VideoCapture('D:/python projects/ai cv/gym traniner/squatsv3.mp4')
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #image = cv2.resize(image, 480,640)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        try:
            landmarks = results.pose_landmarks.landmark
            normalized_landmarks = normalize_landmarks(landmarks)
            #print(normalized_landmarks)
            k =  cv2.waitKey(1)
            if k == 117:
                export_landmark(normalized_landmarks,'up')
            if k == 100:
                export_landmark(normalized_landmarks,'down')
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                    )
            
            #print(image.shape)
            #print(topandbottom(landmarks))
            #topandbottom(landmarks,resolution)
        except Exception as e:
            print(e)
            #break
            #pass
        
        #print(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value])
          
        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        
    



    cap.release()
    cv2.destroyAllWindows()


## train squats model

In [ ]:
df = pd.read_csv('squatcoords.csv')

# Separate features and labels
X = df.iloc[:, 1:]  # Assuming the features start from column 1
y = df['class']

# Encode categorical labels ('up' and 'down') into numerical values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the Neural Network Model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification, so using sigmoid activation
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the Model
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2)

# Evaluate the Model
_, accuracy = model.evaluate(X_test, y_test)
print(f'Model Accuracy: {accuracy * 100:.2f}%')

# Make Predictions
predictions = model.predict(X_test)
model.save('model2.h5')

## Test squats

In [ ]:
model = keras.models.load_model('squatwideornarrow.h5')
import numpy as np

cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #image = cv2.resize(image, 480,640)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        try:
            landmarks = results.pose_landmarks.landmark
            normalized_landmarks = normalize_landmarks(landmarks)
            testdata = np.array([normalized_landmarks])
            predictions = model.predict(testdata)
            #predicted_classes = (predictions > 0.5).astype(int)

            predicted_classes = np.argmax(predictions, axis=1)
            predicted_classes_labels = label_encoder.inverse_transform(predicted_classes.flatten())
            print(f'Predicted Class: {predicted_classes_labels}')
            
            #print(image.shape)
            #print(topandbottom(landmarks))
            #topandbottom(landmarks,resolution)
        except Exception as e:
            print(e)
            #pass
        
        #print(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value])
          
        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        
    


    print('releasing')
    cap.release()
    cv2.destroyAllWindows()


## training model to detect if squats are too wide or narrow 
### exporting landmark

In [ ]:
def make_landmark_list(landmarks):
     new_landmarks = []
     for landmark in landmarks:
          new_landmarks.append(landmark.x)
          new_landmarks.append(landmark.y)
     return new_landmarks
cap = cv2.VideoCapture('squatslegswideornarrow.mp4')
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #image = cv2.resize(image, 480,640)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        try:
            landmarks = results.pose_landmarks.landmark
            #normalized_landmarks = normalize_landmarks(landmarks)
            #print(normalized_landmarks)
            k =  cv2.waitKey(1)
            if k == 111:
                landmarks = make_landmark_list(landmarks)
                export_landmark(landmarks,'perfect')
            if k == 110 :
                landmarks = make_landmark_list(landmarks)
                export_landmark(landmarks,'narrow')
            if k == 119 :
                 landmarks = make_landmark_list(landmarks)
                 export_landmark(landmarks,'wide')
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                    )
            
            #print(image.shape)
            #print(topandbottom(landmarks))
            #topandbottom(landmarks,resolution)
        except Exception as e:
            print(e)
            #break
            #pass
        
        #print(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value])
          
        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        
    



    cap.release()
    cv2.destroyAllWindows()


In [ ]:
df = pd.read_csv('squatsv2narroworwide.csv')

# Separate features and labels
X = df.iloc[:, 1:]  # Assuming the features start from column 1
y = df['class']

# Encode categorical labels ('up' and 'down') into numerical values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the Neural Network Model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='softmax')  # softmax for multiple classes
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the Model
model.fit(X_train, y_train, epochs=60, batch_size=32, validation_split=0.2)

# Evaluate the Model
_, accuracy = model.evaluate(X_test, y_test)
print(f'Model Accuracy: {accuracy * 100:.2f}%')

# Make Predictions
predictions = model.predict(X_test)
#model.save('squatwideornarrow.h5')

In [ ]:
df = pd.read_csv('squatswideornarrow.csv')

# Keep only the desired columns (class, x32, y32, x33, y33)
columns_to_keep = ['class', 'x32', 'y32', 'x33', 'y33']
df_subset = df[columns_to_keep]

# Save the subset to a new CSV file if needed
df_subset.to_csv('squatsv2narroworwide.csv', index=False)


In [ ]:
df = pd.read_csv('squatsv2narroworwide.csv')

# Separate features and labels
X = df.iloc[:, 1:]  # Assuming the features start from column 1
y = df['class']

# Encode categorical labels into numerical values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert numerical labels to one-hot encoding
y_one_hot = keras.utils.to_categorical(y_encoded, num_classes=3)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# Build the Neural Network Model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(3, activation='softmax')  # Three classes, so using softmax activation
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the Model
model.fit(X_train, y_train, epochs=64, batch_size=32, validation_split=0.2)

# Evaluate the Model
_, accuracy = model.evaluate(X_test, y_test)
print(f'Model Accuracy: {accuracy * 100:.2f}%')

In [ ]:
model.save('squatwideornarrowv2.h5')

In [ ]:
model = keras.models.load_model('squatwideornarrow.h5')
import numpy as np

cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #image = cv2.resize(image, 480,640)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        try:
            landmarks = results.pose_landmarks.landmark
            #normalized_landmarks = normalize_landmarks(landmarks)
            normalized_landmarks = [value - landmarks[0] for i, value in enumerate(landmarks) if i not in [1, 3]]

            testdata = np.array([normalized_landmarks])
            #print(testdata)
            predictions = model.predict(testdata)
            #predicted_classes = (predictions > 0.5).astype(int)

            predicted_classes = np.argmax(predictions, axis=1)
            predicted_classes_labels = label_encoder.inverse_transform(predicted_classes.flatten())
            print(f'Predicted Class: {predicted_classes_labels}')
            
            #print(image.shape)
            #print(topandbottom(landmarks))
            #topandbottom(landmarks,resolution)
        except Exception as e:
            print(e)
            #pass
        
        #print(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value])
          
        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        
    


    print('releasing')
    cap.release()
    cv2.destroyAllWindows()


In [ ]:
df = pd.read_csv('squatsv2narroworwide.csv')

# Remove the 3rd and 5th columns
df = df.drop(columns=[df.columns[2], df.columns[4]])

# Subtract the 2nd column from itself and the 3rd column
df[df.columns[1]] = df[df.columns[1]] - df[df.columns[1]]
df[df.columns[2]] = df[df.columns[2]] - df[df.columns[1]]

# Save the modified DataFrame to a new CSV file
df.to_csv('squatsv3narroworwide.csv', index=False)

In [ ]:
df = pd.read_csv('squatsv4wideornarrow.csv')  # Replace 'modified_file.csv' with the actual file path

# Filter rows with specific classes ('perfect', 'wide', 'narrow')
selected_classes = ['perfect', 'wide', 'narrow']
df_filtered = df[df['class'].isin(selected_classes)]

# Group by 'class' and find the lowest and highest values for each column
result = df_filtered.groupby('class').agg({'x1': ['min', 'max'], 'x2': ['min', 'max']})

# Display the result
print(result)

In [ ]:
df = pd.read_csv('try2squatswideornarrow.csv')

# Remove the 3rd and 5th columns
#df = df.drop(columns=[df.columns[2], df.columns[4]])

columns_to_keep = ['class', 'x30', 'x31', 'x32', 'x33']
df_subset = df[columns_to_keep]
df['legonedifference'] = df['x31'] - df['x33']
df['legtwodifference'] = df['x32'] - df['x30']
columns_to_keep = ['class', 'legonedifference', 'legtwodifference']
df_subset = df[columns_to_keep]
df_subset.to_csv('squatsv7wideornarrow.csv')

In [ ]:
def isnarroworwide(feetlandmarks):
    #feetlandmarks  = feetlandmarks[-4:]
    legonedifference = feetlandmarks[1] - feetlandmarks[3]
    legtwodifference = feetlandmarks[2] - feetlandmarks[0]
    #print(legonedifference, ' ', legtwodifference)
    if (legonedifference > -0.01 and legonedifference < 0.01) or (legtwodifference > -0.01 and legtwodifference < 0.01) :
        status = 'perfect'
    elif legonedifference > 0.01 or legtwodifference > 0.01:
        status = 'wide'
    elif legonedifference < -0.01 or legtwodifference < -0.01:
        status = 'narrow'

    else:
        status = 'NA'
    return status
    

In [ ]:
def are_legs_too_open_or_closed(landmarks):
    diff1 = landmarks[30] - landmarks[12]
    diff2 = landmarks[11] -  landmarks[29]
    if (diff1 > -0.015 and diff1 < 0.015) or (diff2 > -0.015 and diff2 < 0.015):
        status = 'perfect'
    elif diff1< -0.015 or diff2< -0.015:
        status = 'too open'
    elif diff1 > 0.015 or diff2< 0.015:
        status = 'too closed'
    else:
        status = 'NA'
    
    return status

In [17]:
model = keras.models.load_model('model2.h5')
import numpy as np
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(['up', 'down'])
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #image = cv2.resize(image, 480,640)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        try:
            landmarks = results.pose_landmarks.landmark
            
            normalized_landmarks = normalize_landmarks(landmarks)
            testdata = np.array([normalized_landmarks])
            predictions = model.predict(testdata)
            predicted_classes = (predictions > 0.5).astype(int)

            #predicted_classes = np.argmax(predictions, axis=1)
            predicted_classes_labels = label_encoder.inverse_transform(predicted_classes.flatten()).tolist()[0]
            landmarks = [landmark.x for landmark in landmarks]
            print(f'Predicted Class: {predicted_classes_labels}')  
            #normalized_landmarks = normalize_landmarks(landmarks)
            #print(isnarroworwide(landmarks[-4:]))
            narroworwide = isnarroworwide(landmarks[-4:])
            too_open_or_closed = are_legs_too_open_or_closed(landmarks)
            # diff1 = landmarks[30] - landmarks[12]
            # diff2 = landmarks[11] -  landmarks[29] 
            #print(diff1, 'second: ', diff2)
            # cv2.putText(image, f'''second:{diff2}''',(50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(image, f'norw: {narroworwide}, oc: {too_open_or_closed}, pos:{predicted_classes_labels}',(50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            #print(image.shape)
            #print(topandbottom(landmarks))
            #topandbottom(landmarks,resolution)
        except Exception as e:
            print(e)
            #pass
        
        #print(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value])
          
        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        
    


    print('releasing')
    cap.release()
    cv2.destroyAllWindows()


1/1 [==============================] - 0s 149ms/step
Predicted Class: down
1/1 [==============================] - 0s 50ms/step
Predicted Class: down
1/1 [==============================] - 0s 93ms/step
Predicted Class: down
1/1 [==============================] - 0s 34ms/step
Predicted Class: down
1/1 [==============================] - 0s 54ms/step
Predicted Class: down
1/1 [==============================] - 0s 43ms/step
Predicted Class: down
1/1 [==============================] - 0s 38ms/step
Predicted Class: down
1/1 [==============================] - 0s 33ms/step
Predicted Class: down
1/1 [==============================] - 0s 39ms/step
Predicted Class: down
1/1 [==============================] - 0s 31ms/step
Predicted Class: down
1/1 [==============================] - 0s 43ms/step
Predicted Class: down
1/1 [==============================] - 0s 46ms/step
Predicted Class: down
1/1 [==============================] - 0s 40ms/step
Predicted Class: down
1/1 [==============================] 